In [ ]:
# n02_all.py  高斯？
import os
import sys
import numpy as np
import pandas as pd
from warnings import filterwarnings
filterwarnings('ignore')
import pcntoolkit as ptk
# sys.path.append('H:\workplace')

DATA_PATH = 'E:/study/data/cerebellum/n02'
os.system('mkdir -p {}'.format(DATA_PATH))
os.chdir(DATA_PATH)

# Create Train Data
# RUN SIEMENS
Y_my = pd.read_csv('E:/study/data/cerebellum/combat_cere_848.csv', index_col = 0).iloc[:848, :]
X_my = pd.read_csv('E:/study/data/cerebellum/cere_info0124_848.csv', index_col = 0).iloc[:848, :]

batch_effects_tr = X_my['cohort'].to_numpy(dtype=int)
uni = np.unique(batch_effects_tr)
for index, i in enumerate(uni):
    np.place(batch_effects_tr, batch_effects_tr==i, index)
n_values = np.max(batch_effects_tr) + 1
bf_tr = np.eye(n_values)[batch_effects_tr]

X_my[[i for i in range(len(uni))]] = bf_tr
X_my[['age', 'sex', 'TIV'] + [i for i in range(len(uni))]].to_csv('X_train.csv',
    sep = ' ',
    header = False,
    index = False)
Y_my.to_csv('Y_train.csv',
    sep = ' ',
    header = False,
    index = False)

# create test data
# RUN SIEMENS
Y_my = pd.read_csv('E:/study/data/cerebellum/combat_cere_848.csv', index_col = 0)
X_my = pd.read_csv('E:/study/data/cerebellum/cere_info0124_848.csv', index_col = 0)

batch_effects_ts = X_my['cohort'].to_numpy(dtype=int)
uni = np.unique(batch_effects_ts)
for index, i in enumerate(uni):
   np.place(batch_effects_ts, batch_effects_ts==i, index)
n_values = np.max(batch_effects_ts) + 1
bf_ts = np.eye(n_values)[batch_effects_ts]

X_my[[i for i in range(len(uni))]] = bf_ts
X_my[['age', 'sex', 'TIV']+ [i for i in range(len(uni))]].to_csv('X_test.csv',
    sep = ' ',
   header = False,
    index = False)
Y_my.to_csv('Y_test.csv',
    sep = ' ',
    header = False,
    index = False)

# GPR: estimate forward model
processing_dir = DATA_PATH

respfile = os.path.join(processing_dir, 'Y_train.csv')       # measurements  (eg cortical thickness) of the training samples (columns: the various features/ROIs, rows: observations or subjects)
covfile = os.path.join(processing_dir, 'X_train.csv')        # covariates (eg age) the training samples (columns: covariates, rows: observations or subjects)

testrespfile_path = os.path.join(processing_dir, 'Y_test.csv')       # measurements  for the testing samples
testcovfile_path = os.path.join(processing_dir, 'X_test.csv')        # covariate file for the testing samples

output_path = os.path.join(processing_dir, 'Models/')    #  output path, where the models will be written
log_dir = os.path.join(processing_dir, 'log/')           #
if not os.path.isdir(output_path):
   os.mkdir(output_path)
if not os.path.isdir(log_dir):
   os.mkdir(log_dir)

outputsuffix = '_GPR'      # a string to name the output files, of use only to you, so adapt it for your needs.

ptk.normative.estimate(covfile = covfile,
            respfile = respfile,
            alg = 'gpr',
            log_path = log_dir,
            output_path = output_path,
            testcov = testcovfile_path,
            testresp = testrespfile_path,
            outputsuffix = outputsuffix,
            savemodel = True)
print("n02成功")

In [ ]:
# n03_plot.py 生成画图指标

import os
import sys
import numpy as np
import pandas as pd
import pcntoolkit as ptk
import time
from warnings import filterwarnings
filterwarnings('ignore')

os.system('n03')
for i in range(210):
    NUM = i
    print(NUM)
    # path = "/path/to/folder"
    DATA_PATH = 'E:/nmodel analysis/cortex/Nmodel/n03/{}'.format(str(NUM))
    # new_folder = "new_folder"
    # new_folder_path = os.path.join(path, new_folder)
    # if not os.path.exists(new_folder_path):
    #     os.makedirs(new_folder_path)
    if not os.path.exists(DATA_PATH):
        print('----')
        os.makedirs(DATA_PATH)
        print('666')
    os.chdir(DATA_PATH)

    # RUN SIEMENS
    Y_my = pd.read_csv('E:/nmodel analysis/cortex/Nmodel/combat_data_brain210.csv', index_col = 0).iloc[:849, NUM]
    print(Y_my)
    X_my = pd.read_csv('E:/nmodel analysis/cortex/Nmodel/BN210_thickness_info_plot.csv', index_col = 0).iloc[:849, :]
    print(X_my)

    batch_effects_tr = X_my['cohort'].to_numpy(dtype=int)
    uni = np.unique(batch_effects_tr)
    for index, i in enumerate(uni):
        np.place(batch_effects_tr, batch_effects_tr==i, index)
    n_values = np.max(batch_effects_tr) + 1
    bf_tr = np.eye(n_values)[batch_effects_tr]

    X_my[[i for i in range(len(uni))]] = bf_tr
    X_my[['age', 'sex'] + [i for i in range(len(uni))]].to_csv('X_train.csv',
        sep = ' ',
        header = False,
        index = False)
    Y_my.to_csv('Y_train.csv',
        sep = ' ',
        header = False,
        index = False)
    print('{} Loaded Train Data'.format(time.asctime( time.localtime(time.time()))))

    # create the plot data
    df_plot = pd.DataFrame()
    
    df_plot['age'] = np.tile(np.linspace(3, 18, 30), 2*len(uni))
    df_plot['sex'] = np.tile(np.concatenate((np.ones(30), np.ones(30) + 1), axis = 0), len(uni))
    df_plot['cohort'] = np.repeat(np.array(uni).reshape(-1, 1), 60)
#     df_plot['age'] = np.tile(np.linspace(0, 18, 30), 2*len(uni))
#     df_plot['sex'] = np.tile(np.concatenate((np.ones(30), np.ones(30) + 1), axis = 0), 1)
#     df_plot['cohort'] = np.tile(np.array(uni.flatten()), 60)
    
#     df_plot['age'] = np.tile(np.linspace(0, 18, 30), 2*len(uni))
#     df_plot['sex'] = np.tile(np.concatenate((np.ones(30), np.ones(30) + 1), axis = 0), len(uni))
#     df_plot['cohort'] = np.repeat(np.array(uni).reshape(-1, 1), 60)
#     df_plot['TIV'] = np.ones(2*len(uni)*30)*0.44
    batch_effects_tr = df_plot['cohort'].to_numpy(dtype=int)

    # os.chdir(father_path)
    X_my = df_plot
    batch_effects_ts = X_my['cohort'].copy(deep=True).to_numpy(dtype=int)
    uni = np.unique(batch_effects_ts)
    for index, i in enumerate(uni):
        np.place(batch_effects_ts, batch_effects_ts==i, index)
    n_values = np.max(batch_effects_ts) + 1
    bf_ts = np.eye(n_values)[batch_effects_ts]

    X_my[[i for i in range(len(uni))]] = bf_ts
    X_my[['age', 'sex'] + [i for i in range(len(uni))]].to_csv('X_plot.csv',
        sep = ' ',
        header = False,
        index = False)
    print('{} Loaded Plot Data'.format(time.asctime( time.localtime(time.time()))))

   # GPR: estimate forward model
    processing_dir = DATA_PATH

    # os.chdir(processing_dir)
    respfile = os.path.join(processing_dir, 'Y_train.csv')       # measurements  (eg cortical thickness) of the training samples (columns: the various features/ROIs, rows: observations or subjects)
    covfile = os.path.join(processing_dir, 'X_train.csv')        # covariates (eg age) the training samples (columns: covariates, rows: observations or subjects)
    testcovfile_path = os.path.join(processing_dir, 'X_plot.csv')        # covariate file for the testing samples
    outputsuffix = '_Plot'      # a string to name the output files, of use only to you, so adapt it for your needs.

    ptk.normative.estimate(covfile = covfile,
                respfile = respfile,
                alg = 'gpr',
                # log_path = log_dir,
                # output_path = output_path,
                testcov = testcovfile_path,
                # testresp = testrespfile_path,
                outputsuffix = outputsuffix,
                savemodel = True)    
print("n03成功")